In [1]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup as BS
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

In [2]:
BASE_URL = "https://www.imdb.com/search/title/?title_type=feature"

In [27]:
webdriver.ChromeOptions().add_argument('--headless')
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(BASE_URL)

In [105]:
page_source = driver.page_source
soup = BS(page_source, "html.parser")

In [4]:
import time
for _ in range(30):
    btn = driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div[1]/span/button')
    btn = btn[0]
    driver.execute_script("arguments[0].scrollIntoView();", btn)
    try:
        btn.click()
    except Exception:
        pass
    time.sleep(1)
print('Done')

In [5]:
soup = BS(driver.page_source.encode('utf-8'), "html.parser")

In [6]:
movies_list_ul = soup.find("ul", {"class": "ipc-metadata-list"})

In [7]:
len(movies_list_ul)

In [8]:
movies_list_with_title = []
for i in movies_list_ul:
    title = i.h3.text
    title = (title.split('.')[1]).strip()
    movies_list_with_title.append({'title': title, 'link': i.a['href']})
    

In [9]:
df = pd.DataFrame(movies_list_with_title)

In [10]:
df.to_csv('movies_list_with_title.csv', sep=",", index=False)